# Kiểm tra GPU

In [ ]:
!nvidia-smi

Wed Jul 26 11:27:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Cài đặt và import các thư viện cần thiết

In [ ]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00


In [ ]:
pip install accelerate -U

# Restart runtime rồi chạy code phía dưới

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding, TextClassificationPipeline
from datasets import load_dataset, load_from_disk, load_metric
import numpy as np

# Load model và tokenizer

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("FPTAI/vibert-base-cased", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("FPTAI/vibert-base-cased")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at FPTAI/vibert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Dataset

In [ ]:
# Download dataset
!gdown 1CriqmsH9MzKn9k78x9GvnH5UTAg3GBIF
!gdown 1RyAZ_vpO2sndK9ZTvJLenVMPcnIwJuhh

Downloading...
From: https://drive.google.com/uc?id=1CriqmsH9MzKn9k78x9GvnH5UTAg3GBIF
To: /content/negative_data.csv
100% 864k/864k [00:00<00:00, 124MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RyAZ_vpO2sndK9ZTvJLenVMPcnIwJuhh
To: /content/positive_data.csv
100% 1.12M/1.12M [00:00<00:00, 170MB/s]


In [ ]:
# Load dataset từ file csv
dataset = load_dataset('csv', data_files={'negative_data.csv', 'positive_data.csv'})

In [ ]:
dataset.shuffle(seeds=20)

DatasetDict({
    train: Dataset({
        features: ['Rate', 'Review', 'Label'],
        num_rows: 4407
    })
})

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Rate', 'Review', 'Label'],
        num_rows: 4407
    })
})

In [ ]:
# Xóa cột không cần thiết
dataset = dataset.remove_columns("Rate")

In [ ]:
# Đổi tên các cột
dataset = dataset.rename_column("Review", "text")
dataset = dataset.rename_column("Label", "label")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4407
    })
})

In [ ]:
dataset = dataset['train']
def strip_text(example):
    # Xóa đi emoji :D :v
    # example["text"].replace(':D', '')
    example["text"] = example["text"].strip()
    return example
dataset = dataset.map(strip_text)



In [ ]:
# Chia dataset thành 2 tập train và test
dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
dataset['train'][0]

{'text': 'Xôi ko có gì đặc giá đắt', 'label': 0}

In [ ]:
# Tokenizer dataset và thêm padding
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=256)
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns='text')
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/3966 [00:00<?, ? examples/s]

Map:   0%|          | 0/441 [00:00<?, ? examples/s]

# Training

In [ ]:
# Viết hàm tính accuracy
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-14-b113ca1a6718>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [ ]:
# Cài đặt các tham số và train
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    # data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.408900,0.245203,0.897959
2,0.209300,0.278467,0.920635
3,0.120800,0.336408,0.918367


TrainOutput(global_step=744, training_loss=0.24634518930988927, metrics={'train_runtime': 554.7562, 'train_samples_per_second': 21.447, 'train_steps_per_second': 1.341, 'total_flos': 1565247668336640.0, 'train_loss': 0.24634518930988927, 'epoch': 3.0})

# Predict

In [ ]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)
pipe("'Biệt đội rất ổn' - phim nhảm đáng quên của Hứa Vĩ Văn, Hoàng Oanh")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[[{'label': 'LABEL_0', 'score': 0.04530138894915581},
  {'label': 'LABEL_1', 'score': 0.9546985626220703}]]

In [ ]:
model.num_parameters()

115355906